# An Adventure Game: Dungeon Dice Monsters

We're going to write the basics of a text-based adventure game.

First we're going to "steal" our Die class from the OOP notebook. **Note**: this could be put in a different file and imported.

In [3]:
import die

test_die = die.Die()
test_die()

3

We will define a class `Creature` that will be the parent of `Player`s and `Monster`s.

In [4]:
DEF_HEALTH = 12
DEF_STRENGTH = die.DEF_FACES


class Creature(object):
    '''This class will be the parent of Player and Monster.
       Creatures will have a name, a location (?),
       health rating, and strength rating.
    '''
    def __init__(self, name, health=DEF_HEALTH, strength=DEF_STRENGTH):
        '''Initialize a Creature.'''
        self.name = name
        self.health = health
        self.strength = strength
        self.alive = True
        self.dungeon = None
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return "I am {} with health of {} and strength of {}".format(self.name,
                                                                     self.health,
                                                                     self.strength)
    
    def is_hit(self, damage):
        '''Suffer a blow from a foe.'''
        self.health -= damage
        if self.health <= 0:
            self.alive = False
            
    def reveal(self):
        if self.dungeon is None:
            print("You are a real nowhere creature!")
        else:
            print(self.dungeon.get_monsters(self))
            
    def locate(self, dungeon):
        self.dungeon = dungeon

In [11]:
cr1 = Creature("John Li")
print(cr1)
cr1.is_hit(4)
cr1.is_hit(4)
print(repr(cr1))
cr1.reveal()

John Li
I am John Li with health of 4 and strength of 6
You are a real nowhere creature!


Let's set up the descendant classes of `Creature`:

In [5]:
class Monster(Creature):
    '''A dangerous, dice-rolling monster!'''
    def __init__(self, name, health=DEF_HEALTH, strength=DEF_STRENGTH):
        super().__init__(name, health=health, strength=strength)
        self.die = die.Die(strength)
        
    def attack(self):
        '''Attack a foe with a certain force.'''
        return self.die()

In [9]:
blob = Monster("The Blob")
print("Blob attack:", blob.attack())
blob.is_hit(2)
print(blob)
print(repr(blob))
dark_magic = Monster("Dark Magician")
blue_eyes = Monster("Endless Dragon with Blue Eyes")

Blob attack: 1
The Blob
I am The Blob with health of 10 and strength of 6


In [7]:
ATTACK_RATIO = 2
MAGE_DISC = 1

class Player(Creature):
    '''A heroic adventurer facing dice monsters in a dungeon.'''
    def __init__(self, name):
        super().__init__(name)
        
    def attack(self):
        '''Attack a foe with a certain force.'''
        return self.strength // ATTACK_RATIO
    
class Mage(Player):
    '''A mage: can reveal.'''
    def __init__(self, name):
        super().__init__(name)
        
    def attack(self):
        '''Attack a foe with a certain force.'''
        return super().attack() - MAGE_DISC

        
class Healer(Player):
    '''A healer: can increase health of teammates.'''
    def __init__(self, name):
        super().__init__(name)

In [10]:
michael = Player("Michael Lippe")
print(repr(michael))
print("Michael attack:", michael.attack())
pablo = Player("Pablo Parra")
somya = Mage("Somya Mehta")
yaseen = Player("Yaseen Elsebaie")
hailey = Player("Hailey Lint")

I am Michael Lippe with health of 12 and strength of 6
Michael attack: 3


In [27]:
DEF_HEIGHT = 10
DEF_WIDTH = 10


class Dungeon(object):
    '''A place for monsters and players to roam and battle.'''
    def __init__(self, name, width=DEF_WIDTH, height=DEF_HEIGHT):
        self.name = name
        self.width = width
        self.height = height
        self.creatures = {}  # dict for now!

    def __getitem__(self, name):
        return self.creatures[name]

    def __str__(self):
        return self.name

    def __repr__(self):
        inhabs = ''
        for name in self.creatures:
            inhabs += "\t{}\n".format(name)
        return str(self) + "\nInhabitants:\n" + inhabs
    
    def get_type(self, creat_type):
        '''Get all creatures of `type` in this dungeon.'''
        # list compr: [expr for condition]
        return [creat for creat in self.creatures.values()
                if isinstance(creat, creat_type)]
        
    def get_monsters(self, creat):
        '''Get all monsters in this dungeon.'''
        if isinstance(creat, Mage):
            return self.get_type(Monster)
        else:
            print("You have no power to reveal what lurks in the darkness!")
            return []
    
    def get_players(self):
        '''Get all players in this dungeon.'''
        return self.get_type(Player)
    
    def add_creat(self, creat):
        '''Add a creature to this object.
        Should we allow re-adding?'''
        self.creatures[str(creat)] = creat
        creat.locate(self)
    
    def meet(self, creat1, creat2):
        '''Two denizens of the dungeon confront each other.'''
        if str(creat1) not in self.creatures:
            print("{} is not in the {}".format(str(creat1), str(self)))
        elif str(creat2) not in self.creatures:
            print("{} is not in the {}".format(str(creat2), str(self)))
        elif isinstance(creat1, Player) and isinstance(creat2, Player):
            print("Hi {} I am {}".format(str(creat1), str(creat2)))
        elif isinstance(creat1, Monster) and isinstance(creat2, Monster):
            print("Aaargh! Growl!")
        else:
            print("{} is fighting {}".format(str(creat1), str(creat2)))
            creat1.is_hit(creat2.attack())
            creat2.is_hit(creat1.attack())

In [28]:
catacomb = Dungeon("Catacomb of Horrors")
catacomb.add_creat(blob)
catacomb.add_creat(michael)
catacomb.add_creat(somya)
catacomb.add_creat(dark_magic)
catacomb.add_creat(pablo)
catacomb.add_creat(hailey)
catacomb.add_creat(blue_eyes)
print("String version:", catacomb)
print(repr(catacomb))
catacomb["The Blob"]

String version: Catacomb of Horrors
Catacomb of Horrors
Inhabitants:
	The Blob
	Michael Lippe
	Somya Mehta
	Dark Magician
	Pablo Parra
	Hailey Lint
	Endless Dragon with Blue Eyes



I am The Blob with health of 10 and strength of 6

In [24]:
# print(catacomb.get_monsters())
print([str(player) for player in catacomb.get_players()])

['Michael Lippe', 'Somya Mehta', 'Pablo Parra', 'Hailey Lint']


In [ ]:
catacomb.meet(blob, michael)
# debug prints:
# print(repr(blob))
# print(repr(michael))

In [29]:
michael.reveal()

You have no power to reveal what lurks in the darkness!
[]


In [30]:
somya.reveal()

[I am The Blob with health of 10 and strength of 6, I am Dark Magician with health of 12 and strength of 6, I am Endless Dragon with Blue Eyes with health of 12 and strength of 6]


In [ ]:
catacomb.meet(pablo, michael)

In [25]:
catacomb.meet(blob, yaseen)

Yaseen Elsebaie is not in the Catacomb of Horrors
